In [1]:
import requests
import bs4
from bs4 import BeautifulSoup as soup
import json
import pandas as pd
from pandas import DataFrame

import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_sm')
from string import punctuation

def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    page = soup(req.text, parser)
    return json.loads("".join(page.find("script", {"type":"application/ld+json"}).contents))

url1 = "https://www.allrecipes.com/recipe/232549"
url2 = "https://www.allrecipes.com/recipe/267555/"
url3 = "https://www.allrecipes.com/recipe/78299/"
url4 = "https://www.allrecipes.com/recipe/13062/"

def get_ingredientScript(url):
    jsonld = get_ld_json(url)
    useful = jsonld[1]
    ingredients = useful["recipeIngredient"]
    return ingredients

In [2]:
def parse_ingredients(url):
    ingredients = get_ingredientScript(url)
    df = DataFrame (ingredients,columns=['ingredients'])
    # 2 cases
    # "number" ("()") ("unit") (adjective) "noun/subject - ingredient" (, other)
    # contains "to taste"
    df = df["ingredients"]
    df_taste = df[df.str.contains('to taste', case = False)]
    
    df_unit = df[~df.str.contains('to taste', case = False)]
    
    # array of arrays: each array is amount, unit, ingredient, descriptor, preparation
    ingredients_parsed = []
    
    for i in df:
        curr_arr = ["", "", "", "", ""]
        print(i)
        if 'to taste' not in i:
        
            # before we look at POS, remove everything after the comma and put it in prep
            split_string = i.split(", ", 1)
            root_phrase = split_string[0]
            if len(split_string) > 1:
                other_piece = split_string[1]
            else:
                other_piece = ""

            curr_arr[4] = other_piece

            split_string2 = root_phrase.split("(", 1)
            if len(split_string2) > 1:
                split_string3 = split_string2[1].split(")", 1)
                curr_arr[3] = split_string3[0]
                root_phrase = split_string2[0].strip() + split_string3[1]
            doc = nlp(root_phrase)
            index = 0
            for token in doc:
                found_num = False
                # only get first number if it matches criteria
                #if found_num == False and token.pos_ == "NOUN" and not token.is_alpha or token.pos_ == "NUM":
                if index == 0:
                    curr_arr[0] = token.text
                elif index == 1:
                    if token.pos_ != "ADJ":
                        curr_arr[1] = token.text
                    else:
                        curr_arr[3] = token.text
                elif token.dep_ == "ROOT":
                    curr_arr[2] = token.text
                else: 
                    curr_arr[3] = curr_arr[3] + " " + token.text
                    curr_arr[3] = curr_arr[3].strip()
                index+=1
        else:
            i = i.replace("to taste", "")
            doc = nlp(i)
            curr_arr[0] = "to taste"
            for token in doc:
                if token.dep_ == "ROOT":
                    curr_arr[2] = token.text
                else:
                    curr_arr[3] = curr_arr[3] + " " + token.text
                    curr_arr[3] = curr_arr[3].strip()
        ingredients_parsed.append(curr_arr)
        #print(token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop, token.children, token.head)
        #displacy.render(doc, style="dep") # change to serve when we go to python


    
    return ingredients_parsed

In [3]:
urls = [url1, url2, url3, url4]
for url in urls:
    print(parse_ingredients(url))

¼ cup butter
½ small white onion, finely diced
3 small zucchinis, diced
3 ears corn, husks and silk removed
sea salt to taste
freshly ground black pepper to taste
[['¼', 'cup', 'butter', '', ''], ['½', '', 'onion', 'small white', 'finely diced'], ['3', '', 'zucchinis', 'small', 'diced'], ['3', 'ears', 'corn', '', 'husks and silk removed'], ['to taste', '', 'salt', 'sea', ''], ['to taste', '', 'pepper', 'freshly ground black', '']]
2 large eggs
⅓ cup milk
1 tablespoon butter, melted
1 teaspoon vanilla extract
1 teaspoon ground cinnamon
4 slices day-old bread, cut into thirds
1 teaspoon confectioners' sugar, or to taste
[['2', '', 'eggs', 'large', ''], ['⅓', 'cup', 'milk', '', ''], ['1', 'tablespoon', 'butter', '', 'melted'], ['1', 'teaspoon', 'extract', 'vanilla', ''], ['1', 'teaspoon', 'cinnamon', 'ground', ''], ['4', 'slices', 'bread', 'day - old', 'cut into thirds'], ['to taste', '', 'sugar', "1 teaspoon confectioners ' , or", '']]
2 pounds ground beef chuck
1 pound bulk Italian saus

In [4]:
vegetarian_dict = {'beef':'beyond beef', 'chicken':'tofu', 'fish':'Gardein fishless filet', 'lamb':'tofu', 'duck':'tofu', 'pork':'tofu',
                   'turkey':'tofu', 'meat':'tofu', 'veal':'tempeh', 'ham':'tofu', 'liver':'black beans',
                   'goose':'tofu', 'salami':'jackfruit', 'bologna':'jackfruit', 'salmon':'marinated carrots',
                   'mahi mahi':'Gardein fishless filet','tilapia':'Gardein fishless filet',
                   'yellowtail':'Gardein fishless filet', 'trout':'Gardein fishless filet',
                   'cod':'Gardein fishless filet', 'venison': 'tofu',
                   'lobster':'Gardein fishless filet', 'shrimp':'Gardein fishless filet', 'crab':'Gardein fishless filet',
                   'eel':'edamame', 'meatball':'chickpea',
                   'sausage':'Beyond sausage', 'bratwurst':'jackfruit', 'hotdog':'Beyond sausage',
                  'hamburger':'impossible burger', 'kielbasa':'chickpeas', 'bacon':'Eggplant', 'chuck':'Beyond beef'}

In [5]:
def to_vegetarian (ingredients):
    for i in ingredients:
        curr_ingredient = i[2]
        if curr_ingredient in vegetarian_dict.keys():
            subst = vegetarian_dict[curr_ingredient]
            subst = subst.strip()
            i[2] = subst
        curr_descr = i[3]
        if curr_descr in vegetarian_dict.keys():
            subst = vegetarian_dict[curr_descr]
            subst = subst.strip()
            i[3] = subst
    print(ingredients)
    return ingredients
        
fake = parse_ingredients(url4)       
to_vegetarian(fake)

½ pound ground beef
¾ cup chopped onion
¾ cup shredded carrots
¾ cup chopped celery
1 teaspoon dried basil
1 teaspoon dried parsley
4 tablespoons butter
3 cups chicken broth
4 cups cubed potatoes
¼ cup all-purpose flour
2 cups cubed Cheddar cheese
1 ½ cups milk
¼ cup sour cream
[['½', 'pound', 'beyond beef', 'ground', ''], ['¾', 'cup', 'chopped', 'onion', ''], ['¾', 'cup', 'carrots', 'shredded', ''], ['¾', 'cup', 'celery', 'chopped', ''], ['1', 'teaspoon', 'basil', 'dried', ''], ['1', 'teaspoon', 'parsley', 'dried', ''], ['4', 'tablespoons', 'butter', '', ''], ['3', 'cups', 'broth', 'tofu', ''], ['4', 'cups', '', 'cubed potatoes', ''], ['¼', 'cup', 'flour', 'all - purpose', ''], ['2', 'cups', 'cubed', 'Cheddar cheese', ''], ['1', '\u2009', 'cups', '½ milk', ''], ['¼', 'cup', 'cream', 'sour', '']]


[['½', 'pound', 'beyond beef', 'ground', ''],
 ['¾', 'cup', 'chopped', 'onion', ''],
 ['¾', 'cup', 'carrots', 'shredded', ''],
 ['¾', 'cup', 'celery', 'chopped', ''],
 ['1', 'teaspoon', 'basil', 'dried', ''],
 ['1', 'teaspoon', 'parsley', 'dried', ''],
 ['4', 'tablespoons', 'butter', '', ''],
 ['3', 'cups', 'broth', 'tofu', ''],
 ['4', 'cups', '', 'cubed potatoes', ''],
 ['¼', 'cup', 'flour', 'all - purpose', ''],
 ['2', 'cups', 'cubed', 'Cheddar cheese', ''],
 ['1', '\u2009', 'cups', '½ milk', ''],
 ['¼', 'cup', 'cream', 'sour', '']]

In [6]:
healthy_dict = {
    "butter":"almond butter",
    "margarine":"almond butter",
    "sugar":"organic coconut sugar",
    "salt":"Morton salt substitute",
    "frosting":"greek yogurt frosting",
    "chocolate":"carob",
    "cheese":"ricotta",
    "bread":"wheat bread",
    "oil":"oil",
    "cream":"soy milk and olive oil",
    "juice":"coconut water",
    "bacon":"turkey bacon",
    "sausage":"turkey sausage",
    "soda":"sparkling water",
    "mayonnaise":"avocado oil",
    "ranch":"hummus",
    "dressing":"balsamic vinaigrette",
    "cereal":"granola",
    "caramel":"coconut milk caramel",
    "flour":"almond flour",
    "syrup":"honey",
    "eggs":"egg whites",
    "egg":"egg white",
    "pasta":"zucchini noodles",
    "rice":"rice"
}

In [7]:
def to_healthy (ingredients):
    for i in ingredients:
        curr_i = i[2]
        if curr_i in healthy_dict.keys():
            subst = healthy_dict[curr_i]
            subst = subst.strip()
            if curr_i == "cream":
                if "whipped" in i[3]:
                    subst = "low-fat cream cheese"
                    i[3] = i[3].replace("whipped", "")
                elif "sour" in i[3]:
                    subst = "yogurt"
                    i[3] = i[3].replace("sour", "")
                elif "ice" in curr_i[3]:
                    subst = "Greek yogurt"
                    i[3] = i[3].replace("ice", "")
            elif curr_i == "bread" and "white" in i[3]:
                i[3] = i[3].replace("white", "")
            elif curr_i == "butter" and "peanut" in i[3]:
                i[3] = i[3].replace("peanut", "")
            elif curr_i == "oil" and "vegetable" in i[3]:
                subst = "canola oil"
                i[3] = i[3].replace("vegetable", "")
            elif curr_i == "rice" and "white" in i[3]:
                subst = "cauliflower rice"
                i[3] = i[3].replace("white", "")
                
            i[2] = subst
    print(ingredients)
    return ingredients
        
fake = parse_ingredients("https://www.allrecipes.com/recipe/8372/")       
to_healthy(fake)

1 ¾ cups all-purpose flour
2 cups white sugar
¾ cup unsweetened cocoa powder
2 teaspoons baking soda
1 teaspoon baking powder
1 teaspoon salt
2 eggs
1 cup strong brewed coffee
1 cup buttermilk
½ cup vegetable oil
1 teaspoon vanilla extract
[['1', '\u2009', 'cups', '¾ all - purpose flour', ''], ['2', 'cups', 'organic coconut sugar', 'white', ''], ['¾', 'cup', 'powder', 'unsweetened cocoa', ''], ['2', 'teaspoons', '', 'baking soda', ''], ['1', 'teaspoon', 'powder', 'baking', ''], ['1', 'teaspoon', 'Morton salt substitute', '', ''], ['2', 'eggs', '', '', ''], ['1', 'cup', 'coffee', 'strong brewed', ''], ['1', 'cup', 'buttermilk', '', ''], ['½', 'cup', 'canola oil', '', ''], ['1', 'teaspoon', 'extract', 'vanilla', '']]


[['1', '\u2009', 'cups', '¾ all - purpose flour', ''],
 ['2', 'cups', 'organic coconut sugar', 'white', ''],
 ['¾', 'cup', 'powder', 'unsweetened cocoa', ''],
 ['2', 'teaspoons', '', 'baking soda', ''],
 ['1', 'teaspoon', 'powder', 'baking', ''],
 ['1', 'teaspoon', 'Morton salt substitute', '', ''],
 ['2', 'eggs', '', '', ''],
 ['1', 'cup', 'coffee', 'strong brewed', ''],
 ['1', 'cup', 'buttermilk', '', ''],
 ['½', 'cup', 'canola oil', '', ''],
 ['1', 'teaspoon', 'extract', 'vanilla', '']]

In [8]:
#lactose free could be very easy...

In [28]:
# mexican
cuisine_dict = {
    'rice':'rice',
    'chocolate':'Mexican chocolate',
    'butter':'lard',
    'cinnamon':'Mexican cinnamon',
    'oregano': 'Mexican oregano',
    'basil':'cilantro',
    'bread':'tortillas',
    'pepper':'pepper',
    'broccoli':'chili pepper',
    'asparagus':'chili pepper',
    'sprouts':'onion',
    'spinach':'corn',
    'zucchini':'corn',
    'zucchinis':'ears of corn',
    'kale':'black beans',
    'potato':'tomato',
    'potatoes':'tomatoes',
    'cauliflower':'avocado',
    'lemon':'lime',
    'lemons':'limes',
    'celery':'onion',
    'cucumber':'corn',
    'cucumbers':'ears of corn',
    'eggplant':'corn',
    'eggplants':'ears of corn',
    'edamame':'pinto beans',
    'mushroom':'onion',
    'mushrooms':'onions',
    'parsley':'thyme',
    'corriander':'cumin',
    'ranch':'tomatillo',
    'mustard':'salsa',
    'dressing':'chile verde',
    'pork':'carnitas',
    'pasta':'Spanish rice',
    'salmon':'cod',
    'halibut':'mahi mahi',
    'tuna':'cod',
    'beef':'carne asada',
    'lamb':'barbacoa',
    'chuck':'barbacoa',
    'cheese':'queso',
    'chicken':'pollo asada'
}

In [29]:
def to_cuisine (ingredients):
    for i in ingredients:
        curr_i = i[2]
        if curr_i in cuisine_dict.keys():
            subst = cuisine_dict[curr_i]
            subst = subst.strip()
            if curr_i == "pepper" and ('black' in i[3] or i[3].strip() == ""):
                subst = "hot Mexican chili powder"
            elif curr_i == "rice" and "white" in i[3]:
                subst = "Spanish rice"
            elif curr_i == "rice" and "brown" in i[3]:
                subst = "Spanish rice"
            elif curr_i == "sprout" and "brussel" in i[3]:
                subst = "onion"
            i[3] = ""
            i[2] = subst
    print(ingredients)
    return ingredients
        
fake = parse_ingredients(url4)       
to_cuisine(fake)

½ pound ground beef
¾ cup chopped onion
¾ cup shredded carrots
¾ cup chopped celery
1 teaspoon dried basil
1 teaspoon dried parsley
4 tablespoons butter
3 cups chicken broth
4 cups cubed potatoes
¼ cup all-purpose flour
2 cups cubed Cheddar cheese
1 ½ cups milk
¼ cup sour cream
[['½', 'pound', 'carne asada', '', ''], ['¾', 'cup', 'chopped', 'onion', ''], ['¾', 'cup', 'carrots', 'shredded', ''], ['¾', 'cup', 'onion', '', ''], ['1', 'teaspoon', 'cilantro', '', ''], ['1', 'teaspoon', 'thyme', '', ''], ['4', 'tablespoons', 'lard', '', ''], ['3', 'cups', 'broth', 'chicken', ''], ['4', 'cups', '', 'cubed potatoes', ''], ['¼', 'cup', 'flour', 'all - purpose', ''], ['2', 'cups', 'cubed', 'Cheddar cheese', ''], ['1', '\u2009', 'cups', '½ milk', ''], ['¼', 'cup', 'cream', 'sour', '']]


[['½', 'pound', 'carne asada', '', ''],
 ['¾', 'cup', 'chopped', 'onion', ''],
 ['¾', 'cup', 'carrots', 'shredded', ''],
 ['¾', 'cup', 'onion', '', ''],
 ['1', 'teaspoon', 'cilantro', '', ''],
 ['1', 'teaspoon', 'thyme', '', ''],
 ['4', 'tablespoons', 'lard', '', ''],
 ['3', 'cups', 'broth', 'chicken', ''],
 ['4', 'cups', '', 'cubed potatoes', ''],
 ['¼', 'cup', 'flour', 'all - purpose', ''],
 ['2', 'cups', 'cubed', 'Cheddar cheese', ''],
 ['1', '\u2009', 'cups', '½ milk', ''],
 ['¼', 'cup', 'cream', 'sour', '']]